In [2]:
import os

os.environ['SUMO_HOME'] = '/opt/homebrew/opt/sumo/share/sumo'

## Create environment using SUMO-RL

In [1]:
# create a SUMO environment for 4x4 network -> 
env_folder = "data/2x2grid"

multi_agent_env = parallel_env(    
        net_file = os.path.join(env_folder, "2x2.net.xml"),
        route_file = os.path.join(env_folder, "2x2.rou.xml"),
        reward_fn = 'diff-waiting-time',
        observation_class = Grid2x2ObservationFunction, 
        out_csv_name="outputs/2x2grid/ppo", 
        num_seconds=80000)

py-spy record -o profile.svg -- python main.py


NameError: name 'SumoEnvironment' is not defined

In [3]:
%load_ext tensorboard
%reload_ext tensorboard

## Instantiate RL algorithm

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig
import ray
ray.init()

/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-22 09:15:22,530	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-22 09:15:25,875	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-22 09:15:26,154	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rl

Python version:,3.9.10
Ray version:,2.8.0


In [5]:
from ray.tune import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv

env_name = "Multi-agent env"
register_env(
    env_name,
    lambda _: ParallelPettingZooEnv(multi_agent_env)
)

observations = multi_agent_env.reset()

 Retrying in 1 seconds


In [6]:
multi_agent_env.action_spaces

/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:158: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


{'1': Discrete(4), '2': Discrete(4), '5': Discrete(4), '6': Discrete(4)}

In [7]:
multi_agent_env.observation_spaces

/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:144: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(


{'1': Box(0.0, 1.0, (63,), float32),
 '2': Box(0.0, 1.0, (63,), float32),
 '5': Box(0.0, 1.0, (63,), float32),
 '6': Box(0.0, 1.0, (63,), float32)}

(raylet) [2023-11-22 09:15:37,724 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 4974321664; capacity: 494384795648. Object creation will fail if spilling is required.


In [9]:
config = (
    PPOConfig()
    .environment(env=env_name, disable_env_checking=False)
    .rollouts(num_rollout_workers=4, rollout_fragment_length=128)
    .training(
        train_batch_size=512,
        lr=2e-5,
        gamma=0.95,
        lambda_=0.9,
        use_gae=True,
        clip_param=0.4,
        grad_clip=None,
        entropy_coeff=0.1,
        vf_loss_coeff=0.25,
        sgd_minibatch_size=64,
        num_sgd_iter=10,
    )
    .debugging(log_level="ERROR")
    .framework(framework="torch")
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)

In [10]:
from ray import tune
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 100000},
    checkpoint_freq=10,
    local_dir="~/ray_results/" + env_name,
    config=config.to_dict(),
)

2023-11-22 09:16:03,168	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-22 09:16:03,172	WARNING __init__.py:10 -- A3C has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.
2023-11-22 09:16:03,172	WARNING __init__.py:10 -- A2C has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, 

(raylet) [2023-11-22 09:16:08,011 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 4974481408; capacity: 494384795648. Object creation will fail if spilling is required.
(pid=7282) 2023-11-22 09:16:08,114	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.


(PPO pid=7282)  Retrying in 1 seconds
(PPO pid=7282) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
(RolloutWorker pid=7288)  Retrying in 1 seconds
(RolloutWorker pid=7290)  Retrying in 1 seconds


(RolloutWorker pid=7290) 2023-11-22 09:16:15,203	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.
(RolloutWorker pid=7288) 2023-11-22 09:16:15,203	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other p

(RolloutWorker pid=7288) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     


...

Step #45000.00 (0ms ?*RT. ?UPS, TraCI: 31ms, vehicles TOT 17867 ACT 12 BUF 0)             CT 10 BUF 0)             
Step #45000.00 (0ms ?*RT. ?UPS, TraCI: 31ms, vehicles TOT 17867 ACT 14 BUF 0)             CT 10 BUF 0)             
Step #45000.00 (0ms ?*RT. ?UPS, TraCI: 30ms, vehicles TOT 17867 ACT 12 BUF 0)             CT 12 BUF 0)             
Step #45000.00 (0ms ?*RT. ?UPS, TraCI: 30ms, vehicles TOT 17867 ACT 17 BUF 0)             CT 11 BUF 0)             


2023-11-22 09:36:32,927	WARNING experiment_state.py:267 -- The previous sync of the experiment directory to the cloud failed with the error: '/Users/loveen/ray_results/Multi-agent env/PPO' is not in the subpath of '~/ray_results/Multi-agent env/PPO' OR one path is relative and the other is absolute.
Syncing will be retried.
2023-11-22 09:36:32,932	WARNING tune.py:1022 -- Trial Runner checkpointing failed: Uploading the experiment directory from the driver (local path: ~/ray_results/Multi-agent env/PPO) to the the cloud (remote path: /Users/loveen/ray_results/Multi-agent env/PPO) failed. Please check the error message above.
2023-11-22 09:36:33,183	INFO tune.py:1047 -- Total run time: 1230.02 seconds (1229.44 seconds for the tuning loop).


Step #45445.00 (0ms ?*RT. ?UPS, TraCI: 1051ms, vehicles TOT 18099 ACT 21 BUF 0)           T 39 BUF 0)             
Step #45445.00 (1ms ~= 1000.00*RT, ~21000.00UPS, TraCI: 1052ms, vehicles TOT 18099 ACT 21 T 39 BUF 0)             


ValueError: No experiment checkpoint file of form 'experiment_state-*.json' was found at: (local, /Users/loveen/ray_results/Multi-agent env/PPO)
Please check if you specified the correct experiment path, which should be a combination of the `storage_path` and `name` specified in your run.

(raylet) [2023-11-22 09:36:38,558 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 3781615616; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-22 09:36:48,630 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 3781492736; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-22 09:36:58,694 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 3781234688; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-22 09:37:08,768 E 7249 83408] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-22_09-15-26_163949_7229 is over 95% full, available space: 3781226496; capacity: 494384795648. Object creation will fail i